In [3]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [4]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

In [5]:

def load_sequence(file_path):
    """Loads a sequence from a FASTA file."""
    with open(file_path, 'r') as file:
        next(file)  # Skip header
        return ''.join(line.strip() for line in file)

def load_bwt(file_path):
    """Loads the last column of the BWT from file."""
    with open(file_path, 'r') as file:
        return file.read().replace('\n', '')

def load_bwt_map(file_path):
    """Loads the BWT mapping from file."""
    with open(file_path, 'r') as file:
        return [int(line.strip()) for line in file]

#______________________________________________________________
reference_sequence = load_sequence('/content/drive/MyDrive/chrX_bwt/chrX.fa')
bwt_sequence = load_bwt('/content/drive/MyDrive/chrX_bwt/chrX_last_col.txt')
bwt_map = load_bwt_map('/content/drive/MyDrive/chrX_bwt/chrX_map.txt')

print("Data Loaded Successfully")
print("Reference Sequence Length:", len(reference_sequence))
print("BWT Sequence Length:", len(bwt_sequence))
print("BWT Map Length:", len(bwt_map))

Data Loaded Successfully
Reference Sequence Length: 151100560
BWT Sequence Length: 151100561
BWT Map Length: 151100561


In [6]:
import pandas as pd
import numpy as np

# Load the data
file_path = "/content/drive/MyDrive/data_list.csv"
data = pd.read_csv(file_path)



In [7]:
data.head()

,errs,pos,meth,string
0,Nil,Nil,Nil,Nil
1,2,255609,mismatch,40;61
2,Nil,Nil,Nil,Nil
3,1,445714,mismatch,67
4,Nil,Nil,Nil,Nil


In [8]:

# Filter out discarded rows (where any of the key columns is `Nil`)
filtered_data = data[(data['errs'] != 'Nil') & (data['pos'] != 'Nil') & (data['meth'] != 'Nil')]

# Use `.loc` to modify columns safely
filtered_data.loc[:, 'errs'] = filtered_data['errs'].astype(int)
filtered_data.loc[:, 'pos'] = filtered_data['pos'].astype(int)

print(f"Number of valid entries: {len(filtered_data)}")

Number of valid entries: 2823573


In [ ]:
from tqdm import tqdm
import numpy as np

# Assuming reference length is known (from chrX.fa)
ref_length = len(reference_sequence)
coverage = np.zeros(ref_length)

# Update coverage array with tqdm progress bar
print("Updating coverage array...")
for _, row in tqdm(filtered_data.iterrows(), total=len(filtered_data), desc="Processing reads"):
    start_pos = row['pos']
    method = row['meth']
    error_positions = str(row['string']).split(';') if pd.notna(row['string']) else []

    # Calculate the end position of the read
    if method == 'mismatch':
        end_pos = start_pos + 100  # Assuming read length is approximately 100
    elif method == 'insdel':
        # Calculate end position considering insertion/deletion adjustments
        ins_count = len(error_positions) // 2  # First half are insertions, second half are deletions
        end_pos = start_pos + 100 + ins_count

    # Update coverage from start to end position
    coverage[start_pos:end_pos] += 1

print("Coverage array updated.")


Updating coverage array...


Processing reads:  58%|█████▊    | 1636142/2823573 [02:17<01:26, 13786.72it/s]

In [ ]:

# Calculate mean and standard deviation of coverage
mean_coverage = np.mean(coverage)
std_coverage = np.std(coverage)
threshold = mean_coverage + 1.5 * std_coverage

# Identify high-coverage regions (potential exons) with progress bar
exon_regions = []
in_exon = False
start = 0

print("Identifying high-coverage regions (potential exons)...")
for i in tqdm(range(ref_length), desc="Scanning coverage array"):
    if coverage[i] >= threshold:
        if not in_exon:
            start = i
            in_exon = True
    else:
        if in_exon:
            exon_regions.append((start, i - 1))
            in_exon = False

# Handle case where exon extends to the end
if in_exon:
    exon_regions.append((start, ref_length - 1))

print(f"Identified {len(exon_regions)} high-coverage regions (potential exons).")

In [ ]:
# Save high-coverage regions to a file
with open("predicted_exons_updated.txt", "w") as exon_file:
    exon_file.write("Start\tEnd\n")
    for start, end in exon_regions:
        exon_file.write(f"{start}\t{end}\n")

print("High-coverage regions saved to 'predicted_exons_updated.txt'.")

# Visualization
import matplotlib.pyplot as plt

plt.figure(figsize=(12, 6))
plt.plot(coverage, color='blue', label='Coverage')
for start, end in exon_regions:
    plt.axvspan(start, end, color='green', alpha=0.3, label='Predicted Exon' if start == exon_regions[0][0] else "")

plt.xlabel('Position on X Chromosome')
plt.ylabel('Coverage')
plt.title('High-Coverage Regions (Exon Prediction)')
plt.legend()
plt.show()
